In [4]:
import duckdb

# 1. Connect (creates argo.db if it doesn’t exist)
con = duckdb.connect("argo.db")
year = 2022

In [2]:
con.execute(f"""
    CREATE TABLE IF NOT EXISTS argo{year}_raw AS
    SELECT *
    FROM read_csv(
        'data/ArgoFloats{year}.csv',
        header = true,
        skip = 1,
        nullstr = ' ',
        columns = {{
            'platform_number': 'VARCHAR',
            'time': 'VARCHAR',
            'latitude': 'VARCHAR',
            'longitude': 'VARCHAR',
            'pres': 'VARCHAR',
            'pres_qc': 'VARCHAR',
            'temp': 'VARCHAR',
            'temp_qc': 'VARCHAR',
            'psal': 'VARCHAR',
            'psal_qc': 'VARCHAR'
        }}
    )
""")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [3]:
con.execute(f"""
CREATE OR REPLACE TABLE argo{year}_slim AS
SELECT
    CAST(platform_number AS INT) AS platform_id,
    CAST(time AS TIMESTAMPTZ) AS date,
    CAST(latitude AS DOUBLE) AS lat,
    CAST(longitude AS DOUBLE) AS lon,
    CAST(pres AS DOUBLE) AS depth_m,
    CAST(temp AS DOUBLE) AS temp_c,
    CAST(psal AS DOUBLE) AS sal_psu,

    -- Only keep digits 09, else NULL
    CAST(NULLIF(regexp_extract(pres_qc, '^[0-9]$', 0), '') AS TINYINT) AS pres_qc,
    CAST(NULLIF(regexp_extract(temp_qc, '^[0-9]$', 0), '') AS TINYINT) AS temp_qc,
    CAST(NULLIF(regexp_extract(psal_qc, '^[0-9]$', 0), '') AS TINYINT) AS psal_qc

FROM argo{year}_raw;

""")


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [4]:
con.execute(f"""delete from argo{year}_slim where 
lat NOT BETWEEN -90 AND 90
OR lon NOT BETWEEN -180 AND 180;""")

In [5]:
con.execute("INSTALL spatial; LOAD spatial;")

In [6]:
con.execute(f"""      
CREATE TABLE argo{year}_positions AS
SELECT DISTINCT
    lat,
    lon,
    ST_Point(lon, lat) AS geom
FROM
    argo{year}_slim;""")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [7]:
# con.execute(f"""
# LOAD spatial;
# CREATE TABLE IF NOT EXISTS world_seas_iho_v3 AS
# SELECT *
# FROM ST_Read('World_Seas_IHO_v3/World_Seas_IHO_v3.shp');""")

In [8]:
# con.execute(f"""
# ALTER TABLE world_seas_iho_v3
# ADD COLUMN geom_simple GEOMETRY;

# UPDATE world_seas_iho_v3
# SET geom_simple = ST_Simplify(geom, 0.01);""")

In [9]:
con.execute(f"""
CREATE INDEX IF NOT EXISTS idx_world_seas_geom ON world_seas_iho_v3 USING rtree(geom_simple);
CREATE INDEX IF NOT EXISTS idx_positions_geom ON argo{year}_positions USING rtree(geom);
ANALYZE world_seas_iho_v3;
ANALYZE argo{year}_positions;
""")

In [10]:
con.execute(f"""
-- First, assign regions where there is an intersection
CREATE TABLE positions_with_region AS
SELECT 
    p.lat,
    p.lon,
    o.name AS region_name,
    p.geom
FROM argo{year}_positions p
LEFT JOIN world_seas_iho_v3 o
    ON ST_Intersects(p.geom, o.geom_simple);
""")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [11]:
con.execute(f"""
-- Next, find the nearest region for points with NULL region_name
CREATE TABLE nearest_region AS
SELECT 
    p.lat,
    p.lon,
    s.name AS region_name
FROM positions_with_region p
JOIN world_seas_iho_v3 s
    ON p.region_name IS NULL
QUALIFY ROW_NUMBER() OVER (PARTITION BY p.lat, p.lon ORDER BY ST_Distance(p.geom, s.geom_simple)) = 1;
""")

In [12]:
con.execute(f"""
-- Finally, combine the tables
CREATE TABLE argo{year}_positions_region AS
SELECT lat, lon, region_name 
FROM positions_with_region
WHERE region_name IS NOT NULL

UNION ALL

SELECT lat, lon, region_name 
FROM nearest_region
WHERE region_name IS NOT NULL;
""")

In [13]:
con.execute(f"""
DROP TABLE argo{year}_positions;
DROP TABLE argo{year}_raw;
DROP TABLE nearest_region;
DROP TABLE positions_with_region;""").fetch_df()

,Success


In [14]:
con.execute(f"""
CREATE TABLE argo{year}_with_region AS
SELECT a.*, r.region_name
FROM argo{year}_slim a
LEFT JOIN argo{year}_positions_region r
ON a.lat = r.lat AND a.lon = r.lon;
""")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [15]:
con.execute(f"DROP TABLE argo{year}_slim").fetch_df()

,Success


In [16]:
con.execute(f"ALTER TABLE argo{year}_with_region RENAME TO argo{year};")

In [17]:
con.execute(f"""Create table distinct_float_positions_{year} as
Select distinct platform_id, date, lat, lon from argo{year};""").fetchdf()

,Count
0,176314


In [19]:
con.execute(f"""CREATE TABLE latest_float_positions_{year} AS
SELECT
    platform_id,
    ARGMAX(lat, date) AS lat,
    ARGMAX(lon, date) AS lon,
    MAX(date) AS date
FROM
    distinct_float_positions_{year}
GROUP BY
    platform_id;""").fetch_df()

,Count
0,4718


In [5]:
con.execute("SHOW TABLES;").fetch_df()

,name
0,argo2022
1,argo2022_positions_region
2,argo2023
3,argo2023_positions_region
4,argo2024
5,argo2024_positions_region
6,distinct_float_positions_2022
7,distinct_float_positions_2023
8,distinct_float_positions_2024
9,latest_float_positions_2022


In [ ]:
con.close()